# Plant Seedlings Classification

### Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Lambda,Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical
import keras
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.optimizers import RMSprop,Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing import image
import os
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


### Dataset Exploration 

In [2]:
data_dir="Dataset/train"
batch_size = 32
img_height = 180
img_width = 180
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 4750 files belonging to 12 classes.
Using 3800 files for training.


In [3]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 4750 files belonging to 12 classes.
Using 950 files for validation.


In [4]:
class_names = train_ds.class_names
print(class_names)

['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent', 'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']


In [5]:
from tensorflow.keras import layers

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

### Understanding our model
- The model we have created is for understanding of all the concepts one would require for understanding and building your deep learning models. The model we have created here uses functional model structure defined in Keras.

In [6]:
num_classes = 12

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])



In [7]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [8]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)                    
# Calling `save('my_model.h5')` creates a h5 file `my_model.h5`.
model.save("plantseeding.h5")

Epoch 1/10
119/119 [==============================] - 253s 2s/step - loss: 2.2467 - accuracy: 0.2199 - val_loss: 1.4786 - val_accuracy: 0.5137
Epoch 2/10
119/119 [==============================] - 120s 990ms/step - loss: 1.3131 - accuracy: 0.5553 - val_loss: 1.2045 - val_accuracy: 0.6000
Epoch 3/10
119/119 [==============================] - 125s 1s/step - loss: 1.0420 - accuracy: 0.6551 - val_loss: 0.9846 - val_accuracy: 0.6811
Epoch 4/10
119/119 [==============================] - 125s 1s/step - loss: 0.8190 - accuracy: 0.7200 - val_loss: 0.9747 - val_accuracy: 0.6821
Epoch 5/10
119/119 [==============================] - 126s 1s/step - loss: 0.6541 - accuracy: 0.7912 - val_loss: 0.9374 - val_accuracy: 0.7063
Epoch 6/10
119/119 [==============================] - 192s 2s/step - loss: 0.5113 - accuracy: 0.8358 - val_loss: 0.7844 - val_accuracy: 0.7579
Epoch 7/10
119/119 [==============================] - 380s 3s/step - loss: 0.4086 - accuracy: 0.8575 - val_loss: 0.8025 - val_accuracy: 0.7

In [9]:
import tensorflow
# It can be used to reconstruct the model identically.
reconstructed_model = tensorflow.keras.models.load_model("plantseeding.h5")

### Prepare testing Dataset and predict the output

In [10]:
import cv2
def image_reader(path):
    t=cv2.imread(path)
    t=cv2.cvtColor(t, cv2.COLOR_BGR2RGB)
    t=cv2.resize(t, (180,180))
    return t

def test_prepare_data(list_of_images,test_dir):
    x=[]
    for image in list_of_images:
        x.append(image_reader(test_dir+image))
    return x
test_dir='Dataset/test/'
test_fnames=os.listdir(test_dir)
testing_data=test_prepare_data(test_fnames,test_dir)
evaluation=np.asarray(testing_data)

In [11]:
predictions=model.predict(evaluation)
predictions=[np.argmax(i) for i in predictions]
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
training_labels=le.fit_transform(class_names)
target=le.inverse_transform(predictions)
print(target)

['Small-flowered Cranesbill' 'Fat Hen' 'Fat Hen' 'Common Chickweed'
 'Sugar beet' 'Loose Silky-bent' 'Common Chickweed' 'Fat Hen'
 'Loose Silky-bent' 'Loose Silky-bent' 'Common Chickweed'
 'Small-flowered Cranesbill' 'Scentless Mayweed' 'Scentless Mayweed'
 'Fat Hen' 'Fat Hen' 'Scentless Mayweed' 'Scentless Mayweed' 'Fat Hen'
 'Charlock' 'Fat Hen' 'Small-flowered Cranesbill' 'Shepherds Purse'
 'Sugar beet' 'Sugar beet' 'Maize' 'Fat Hen' 'Scentless Mayweed'
 'Common Chickweed' 'Sugar beet' 'Common Chickweed' 'Shepherds Purse'
 'Loose Silky-bent' 'Cleavers' 'Loose Silky-bent' 'Loose Silky-bent'
 'Loose Silky-bent' 'Loose Silky-bent' 'Common Chickweed' 'Maize'
 'Small-flowered Cranesbill' 'Fat Hen' 'Common Chickweed'
 'Small-flowered Cranesbill' 'Sugar beet' 'Cleavers' 'Maize'
 'Scentless Mayweed' 'Maize' 'Small-flowered Cranesbill'
 'Loose Silky-bent' 'Sugar beet' 'Cleavers' 'Loose Silky-bent' 'Maize'
 'Fat Hen' 'Charlock' 'Small-flowered Cranesbill' 'Cleavers'
 'Loose Silky-bent' 'Sugar

In [16]:
submission = pd.DataFrame({'Image': test_fnames,'target': target})
submission.to_csv('test.csv', index=False)